In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,KFold, cross_val_score
from sklearn.metrics import mean_absolute_error , r2_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression , Ridge , Lasso, RidgeCV
from sklearn.preprocessing import OneHotEncoder ,RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate

import warnings

warnings.filterwarnings("ignore")

# EDA

In [ ]:
pd.set_option ('display.max_columns', None)
# pd.set_option ('display.max_rows', None)
#pd.set_option ('max_rows', 400)
pd.set_option ('max_colwidth', 400)

In [ ]:
df = pd.read_csv(r"/kaggle/input/route-ai-innovators-challenge/train.csv")

In [ ]:
df

In [ ]:
df.rename(columns = {"Unnamed: 0": "id"},inplace = True)
df.head(5)

In [ ]:
df.describe()

In [ ]:
df.info()

# Handling missing values

In [ ]:
(df.isnull().sum()/8194 *100).sort_values(ascending = False)
#aps, sps, surv2m, surv6m , prg2m, prg6m, dnr, dnrday.

In [ ]:
df["dnr"].mode()

In [ ]:
df["sod"] = df["sod"].fillna(df["sod"].mean())
df["temp"] = df["temp"].fillna(df["temp"].mean())
df["resp"] = df["resp"].fillna(df["resp"].mean())
df["hrt"] = df["hrt"].fillna(df["hrt"].mean())
df["meanbp"] = df["meanbp"].fillna(df["meanbp"].mean())
df["scoma"] = df["scoma"].fillna(df["scoma"].mean())
df["aps"] = df["aps"].fillna(df["aps"].mean())
df["surv2m"] = df["surv2m"].fillna(df["surv2m"].mean())
df["sps"] = df["sps"].fillna(df["sps"].mean())
df["surv6m"] = df["surv6m"].fillna(df["surv6m"].mean())
df["dnr"] = df["dnr"].fillna("no dnr")
df["dnrday"] = df["dnrday"].fillna(df["dnrday"].mean())
df["crea"] = df["crea"].fillna(df["crea"].mean())
df["avtisst"] = df["avtisst"].fillna(df["avtisst"].mean())
df["wblc"] = df["wblc"].fillna(df["wblc"].mean())
df["charges"] = df["charges"].fillna(df["charges"].mean())

## adlp column missing value 62 so we gonna drop it

In [ ]:
df.drop("adlp",axis = 1,inplace = True)
df.drop("urine",axis = 1,inplace = True)
df.drop("glucose",axis = 1,inplace = True)
df.drop("bun",axis = 1,inplace = True)
df.drop("totmcst",axis = 1,inplace = True)
df.drop("alb",axis = 1,inplace = True)
df.drop("income",axis = 1,inplace = True)
df.drop("adls",axis = 1,inplace = True)

In [ ]:
df.columns

In [ ]:
(df.isnull().sum()/8194 *100).sort_values(ascending = False)

In [ ]:
#(df.isnull().sum()/x.shape[0] *100).sort_values(ascending = False)

In [ ]:
sns.countplot(data=df, x='diabetes')

In [ ]:
droped_values =df['bili'][(df['bili'] >= df.bili.quantile(0.25)) & (df['bili'] <= df.bili.quantile(0.75))].dropna()
df["bili"] = df["bili"].fillna(pd.Series(np.random.choice(droped_values, size = len(df.index))))


In [ ]:
droped_values =df['pafi'][(df['pafi'] >= df.pafi.quantile(0.25)) & (df['pafi'] <= df.pafi.quantile(0.75))].dropna()
df["pafi"] = df["pafi"].fillna(pd.Series(np.random.choice(droped_values, size = len(df.index))))

In [ ]:
droped_values =df['ph'][(df['ph'] >= df.ph.quantile(0.25)) & (df['ph'] <= df.ph.quantile(0.75))].dropna()
df["ph"] = df["ph"].fillna(pd.Series(np.random.choice(droped_values, size = len(df.index))))

In [ ]:
droped_values =df['prg2m'][(df['prg2m'] >= df.prg2m.quantile(0.25)) & (df['prg2m'] <= df.prg2m.quantile(0.75))].dropna()
df["prg2m"] = df["prg2m"].fillna(pd.Series(np.random.choice(droped_values, size = len(df.index))))

In [ ]:
droped_values =df['edu'][(df['edu'] >= df.edu.quantile(0.25)) & (df['edu'] <= df.edu.quantile(0.75))].dropna()
df["edu"] = df["edu"].fillna(pd.Series(np.random.choice(droped_values, size = len(df.index))))

In [ ]:
droped_values =df['prg6m'][(df['prg6m'] >= df.prg6m.quantile(0.25)) & (df['prg6m'] <= df.prg6m.quantile(0.75))].dropna()
df["prg6m"] = df["prg6m"].fillna(pd.Series(np.random.choice(droped_values, size = len(df.index))))

In [ ]:
droped_values =df['totcst'][(df['totcst'] >= df.totcst.quantile(0.25)) & (df['totcst'] <= df.totcst.quantile(0.75))].dropna()
df["totcst"] = df["totcst"].fillna(pd.Series(np.random.choice(droped_values, size = len(df.index))))

In [ ]:
df["race"].unique()

In [ ]:
df["race"].value_counts()

In [ ]:
df["race"] = df["race"].fillna("other")

In [ ]:
#(df.isnull().sum()/x.shape[0] *100).sort_values(ascending = False)

In [ ]:
sns.countplot(data = df , x = 'hospdead')

In [ ]:
df.select_dtypes(include='object').columns

In [ ]:
df.columns

# Outliers

In [ ]:
sns.histplot(data = df["age"])


In [ ]:
sns.boxplot(data=df, x = "age")

In [ ]:
sns.histplot(data = df["edu"])

In [ ]:
sns.boxplot(data=df, x = "edu")

In [ ]:
sns.histplot(data=df['charges'], bins=30)  # Adjust the number of bins as needed
plt.xlabel('Charges')
plt.ylabel('Frequency')
plt.title('Distribution of Charges')
plt.xticks(ticks=[df["charges"].min(), df["charges"].max()])
plt.show()

In [ ]:
sns.boxplot(data=df, x = "charges")

In [ ]:
sns.histplot(data = df["hday"], bins = 15)


In [ ]:
sns.boxplot(data=df, x = "hday")

In [ ]:
sns.histplot(data = df["meanbp"])

In [ ]:
sns.boxplot(data=df, x = "meanbp")

In [ ]:
sns.histplot(data = df["wblc"])

In [ ]:
sns.boxplot(data=df, x = "wblc")

In [ ]:
sns.histplot(data = df["hrt"])

In [ ]:
sns.boxplot(data=df, x = "hrt")

In [ ]:
sns.histplot(data = df["resp"])

In [ ]:
sns.boxplot(data=df, x = "resp")

In [ ]:
sns.histplot(data = df["temp"])

In [ ]:
sns.boxplot(data=df, x = "temp")

In [ ]:
sns.histplot(data = df["pafi"])

In [ ]:
sns.boxplot(data=df, x = "pafi")

In [ ]:
sns.histplot(data = df["bili"])

In [ ]:
sns.boxplot(data=df, x = "bili")

In [ ]:
sns.histplot(data = df["crea"])

In [ ]:
sns.boxplot(data=df, x = "crea")

In [ ]:
df.columns

In [ ]:
sns.histplot(data = df["sod"])

In [ ]:
sns.boxplot(data=df, x = "sod")

In [ ]:
sns.histplot(data = df["ph"])

In [ ]:
sns.boxplot(data=df, x = "ph")

In [ ]:
df.columns

In [ ]:
sns.histplot(data = df["sps"])

In [ ]:
sns.boxplot(data=df, x = "sps")

In [ ]:
sns.histplot(data = df["aps"])

In [ ]:
sns.boxplot(data=df, x = "aps")

In [ ]:
sns.histplot(data = df["surv2m"])

In [ ]:
sns.boxplot(data=df, x = "surv2m")

In [ ]:
sns.histplot(data = df["surv6m"])

In [ ]:
sns.boxplot(data=df, x = "surv6m")

In [ ]:
sns.histplot(data = df["prg2m"])

In [ ]:
sns.boxplot(data=df, x = "prg2m")

In [ ]:
sns.histplot(data = df["prg6m"])

In [ ]:
sns.boxplot(data=df, x = "prg6m")

In [ ]:
sns.histplot(data = df["adlsc"])

In [ ]:
sns.boxplot(data=df, x = "adlsc")

In [ ]:
df.describe()

In [ ]:
def outliers(df,ft):
    q1 = df[ft].quantile(0.25)
    q3 = df[ft].quantile(0.75)
    iqr = q3 - q1 
    
    lower_limit = q1 - iqr *1.5
    upper_limit = q3 + iqr *1.5
    
    ls = df.index[(df[ft]<lower_limit) | (df[ft]>upper_limit)]
    
    return ls

In [ ]:
df.columns

In [ ]:
index_list = []
num = [ 'age', 'edu','charges', 'sps', 'aps', 'surv2m','surv6m', 'hday', 'prg2m', 'prg6m',
       'dnrday', 'meanbp', 'wblc', 'hrt', 'resp', 'temp', 'pafi', 'bili',
       'crea', 'sod', 'ph', 'adlsc']
for featsure in num:
    index_list.extend(outliers(df,featsure))

In [ ]:
def remove(df,ls):
    ls = sorted(set(ls))
    df = df.drop(ls)
    return df

In [ ]:
df = remove(df,index_list)

In [ ]:
df.shape

In [ ]:
df.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

In [ ]:
for fature in num:
    print(fature)
    print(df[fature].skew())


In [ ]:
#right medium(adlsc ,temp, aps,wblc)
#right high(sps,charges , hday  , dnday , bili,crea)
#left medium (edu  , surv6m)
#left high(prg2m,surv2m)

In [ ]:
df['adlsc'] = np.sqrt(np.abs(df['adlsc']))
df['temp'] = np.sqrt(np.abs(df['temp']))
df['aps'] = np.sqrt(np.abs(df['aps']))
df['wblc'] = np.sqrt(np.abs(df['wblc']))

In [ ]:
df['sps'] = np.log1p(df['sps'])
df['charges'] = np.log1p(df['charges'])
df['hday'] = np.log1p(df['hday'])
df['bili'] = np.log1p(df['bili'])
df['crea'] =np.log1p(df['crea'])

In [ ]:
df['edu'] = df['edu'].pow(2)
df['surv6m'] = df['surv6m'].pow(2)

In [ ]:
df['prg2m'] = np.exp(df['prg2m'])
df['surv2m'] = np.exp(df['surv2m'])

In [ ]:
cor = df.select_dtypes(include='number').copy()
corr = cor.corr()

fig , ax = plt.subplots(figsize=(25 , 20))


sns.heatmap(corr ,annot= True , ax=ax , cmap= 'Blues');

In [ ]:
print(df['charges'].corr(df['hospdead']))
print(df['totcst'].corr(df['hospdead']))

In [ ]:
print(df['avtisst'].corr(df['hospdead']))
print(df['edu'].corr(df['hospdead']))

In [ ]:
df["dnrday"].unique()

In [ ]:
df.isna().any()

In [ ]:
df['dnrday'].dtype

In [ ]:
df["hospdead"].value_counts()

In [ ]:
df.drop(columns = ["totcst"],axis =1 , inplace = True)


In [ ]:
cols = ['sex', 'dzgroup', 'dzclass',"dnr", 'race', 'ca']
df[cols] = df[cols].apply(LabelEncoder().fit_transform)

In [ ]:
x=df.drop('hospdead',axis=1)
y=df['hospdead']

In [ ]:
sm = SMOTE(random_state=42)
X, Y = sm.fit_resample(x, y)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25,random_state=42)

In [ ]:
Y.mean()

In [ ]:
# (500 , 3, 0.1) = 94,89
# (500 , 5, 0.1) = 99,90

In [ ]:
x_train.shape

In [ ]:
xgb = XGBClassifier(n_estimators= 400 , max_depth= 7 , learning_rate = 0.1)

In [ ]:
xgb.fit(x_train , y_train);

In [ ]:
print (xgb.score(x_train , y_train))
print (xgb.score(x_test , y_test))

In [ ]:
print(classification_report(y_test,xgb.predict(x_test)))

In [ ]:
rf = RandomForestClassifier(n_estimators= 50 , max_depth= 11 , max_features=29)

In [ ]:
rf.fit(x_train , y_train);

In [ ]:
print (rf.score(x_train , y_train))
print (rf.score(x_test , y_test))

In [ ]:
print(classification_report(y_test,rf.predict(x_test)))

# submission

In [ ]:
test = pd.read_csv(r"/kaggle/input/route-ai-innovators-challenge/test.csv")

In [ ]:
test.shape

In [ ]:
test.drop("adlp",axis = 1,inplace = True)
test.drop("urine",axis = 1,inplace = True)
test.drop("glucose",axis = 1,inplace = True)
test.drop("bun",axis = 1,inplace = True)
test.drop("totmcst",axis = 1,inplace = True)
test.drop("alb",axis = 1,inplace = True)
test.drop("income",axis = 1,inplace = True)
test.drop("adls",axis = 1,inplace = True)

In [ ]:
test.drop(columns = ["totcst"],axis =1 , inplace = True)

In [ ]:
test["sod"] = test["sod"].fillna(test["sod"].mean())
test["temp"] = test["temp"].fillna(test["temp"].mean())
test["resp"] = test["resp"].fillna(test["resp"].mean())
test["hrt"] = test["hrt"].fillna(test["hrt"].mean())
test["meanbp"] = test["meanbp"].fillna(test["meanbp"].mean())
test["scoma"] = test["scoma"].fillna(test["scoma"].mean())
test["aps"] = test["aps"].fillna(test["aps"].mean())
test["surv2m"] = test["surv2m"].fillna(test["surv2m"].mean())
test["sps"] = test["sps"].fillna(test["sps"].mean())
test["surv6m"] = test["surv6m"].fillna(test["surv6m"].mean())
test["dnr"] = test["dnr"].fillna("no dnr")
test["dnrday"] = test["dnrday"].fillna(test["dnrday"].mean())
test["crea"] = test["crea"].fillna(test["crea"].mean())
test["avtisst"] = test["avtisst"].fillna(test["avtisst"].mean())
test["wblc"] = test["wblc"].fillna(test["wblc"].mean())
test["charges"] = test["charges"].fillna(test["charges"].mean())

In [ ]:
(test.isnull().sum()).sort_values(ascending = False)

In [ ]:
cols = ['sex', 'dzgroup', 'dzclass', 'dnr', 'race', 'ca']
test[cols] = test[cols].apply(LabelEncoder().fit_transform)

In [ ]:
test.rename(columns = {"Unnamed: 0": "id"},inplace = True)

In [ ]:
droped_values =test['prg6m'][(test['prg6m'] >= test.prg6m.quantile(0.25)) & (test['prg6m'] <= test.prg6m.quantile(0.75))].dropna()
test["prg6m"] = test["prg6m"].fillna(pd.Series(np.random.choice(droped_values, size = len(test.index))))

In [ ]:
droped_values =test['edu'][(test['edu'] >= test.edu.quantile(0.25)) & (test['edu'] <= test.edu.quantile(0.75))].dropna()
test["edu"] = test["edu"].fillna(pd.Series(np.random.choice(droped_values, size = len(test.index))))

In [ ]:
droped_values =test['prg2m'][(test['prg2m'] >= test.prg2m.quantile(0.25)) & (test['prg2m'] <= test.prg2m.quantile(0.75))].dropna()
test["prg2m"] = test["prg2m"].fillna(pd.Series(np.random.choice(droped_values, size = len(test.index))))

In [ ]:
droped_values =test['ph'][(test['ph'] >= test.ph.quantile(0.25)) & (test['ph'] <= test.ph.quantile(0.75))].dropna()
test["ph"] = test["ph"].fillna(pd.Series(np.random.choice(droped_values, size = len(test.index))))

In [ ]:
droped_values =test['pafi'][(test['pafi'] >= test.pafi.quantile(0.25)) & (test['pafi'] <= test.pafi.quantile(0.75))].dropna()
test["pafi"] = test["pafi"].fillna(pd.Series(np.random.choice(droped_values, size = len(test.index))))

In [ ]:
droped_values =test['bili'][(test['bili'] >= test.bili.quantile(0.25)) & (test['bili'] <= test.bili.quantile(0.75))].dropna()
test["bili"] = test["bili"].fillna(pd.Series(np.random.choice(droped_values, size = len(test.index))))

In [ ]:
test["race"] = test["race"].fillna("other")

In [ ]:
(test.isnull().sum()).sort_values(ascending = False)

In [ ]:
test['adlsc'] = np.sqrt(np.abs(test['adlsc']))
test['temp'] = np.sqrt(np.abs(test['temp']))
test['aps'] = np.sqrt(np.abs(test['aps']))
test['wblc'] = np.sqrt(np.abs(test['wblc']))

In [ ]:
test['sps'] = np.log1p(test['sps'])
test['charges'] = np.log1p(test['charges'])
test['hday'] = np.log1p(test['hday'])
test['bili'] = np.log1p(test['bili'])
test['crea'] =np.log1p(test['crea'])

In [ ]:
test['edu'] = test['edu'].pow(2)
test['surv6m'] = test['surv6m'].pow(2)

In [ ]:
test['prg2m'] = np.exp(test['prg2m'])
test['surv2m'] = np.exp(test['surv2m'])

In [ ]:
predections = xgb.predict(test)

In [ ]:
submisson =test[["id"]]

In [ ]:
submisson["hospdead"] = predections

In [ ]:
submisson

In [ ]:
submisson.to_csv("submisson.csv" , index = False)